In [2]:
import pickle
import multiprocessing
import os, re, sys
import nltk
import csv
import gensim.models.word2vec as w2v
import glob2
import time

# word2vec with Reddit corpus

In [3]:
songdata_filepath = "songdata.csv"                #simple csv

reddit_filepath = "reddit-top-2.5-million-master/data" #this is a dir, we'll walk through it to create a
                                                  #dict of subreddits and posts


## just playing arond with etl on the song data

In [4]:
songs_list = []

with open(songdata_filepath, 'rb') as csvfile:
    song_reader = csv.reader(csvfile)
    i = 0
    for row in song_reader:
        if(i == 0):
            next
        else:
            song = {'artist': row[0], 'title': row[1], 'lyric_link': row[2], 'text': row[3]}
            songs_list.append(song)
        i += 1

In [5]:
songs_list[258]

{'artist': 'Air Supply',
 'lyric_link': '/a/air+supply/do+it+again_20004902.html',
 'text': "I used to watch you walk away  \nTil the road you take would bend  \nI was the man to turn you on  \nAnd you knew it after all  \nCome tomorrow when I'm all weary  \nYou would turn me down again  \nAt least that's the way I can remember  \n  \nDo it again, do it again  \nDo it again, do it again  \n  \nEvery little word you told me got home  \nI'm still thinking about them  \nEvery little secret we could have known  \nAnd we can't live without them  \n  \nYou surely started something  \nYou surely gave me something new  \n  \nDo it again, do it again  \nDo it again, do it again  \n  \nAnd I do think about you each now and then  \nAnd each time it gets tighter  \nFrom the road I can see the last break  \nAnd my head's getting lighter  \n  \nDo it again, do it again (repeat)  \n\n",
 'title': 'Do It Again'}

In [6]:
len(songs_list)   #should be 57,650 songs

57650

## Now on to the subreddit posts

In [7]:
subreddit_list = {}

path = reddit_filepath + '/**'
files = glob2.glob(path)

len(files)   #should be 2,499 subreddits

subreddit_id_to_title = {}

In [8]:
j = 0
for file in files:
    subreddit = file.replace(reddit_filepath + "/", "").replace(".csv", "")
    subreddit_id_to_title[j] = subreddit
    j += 1
    with(open(file, 'rb')) as csvfile:
        reddit_reader = csv.reader(csvfile)
        posts = []
        reddit_reader.next()
        for post in reddit_reader:
            posts.append({'score': post[1].decode('utf-8'), 'title': post[4].decode('utf-8'), 'text': post[9].decode('utf-8')})
        
        subreddit_list[subreddit] = posts

In [9]:
subreddit_list["0x10c"][0]

{'score': u'430',
 'text': u"If you saw my tweet yesterday about being single now, that's the reason I haven't had that much inspiration lately to work on a game as big as 0x10c, and hopefully it explains why I haven't spoken much about the reason.\n\nI don't want to sound whiny on twitter, but I thought this excellent subreddit might want to know. I haven't forgotten about you!",
 'title': u'The reason for slow'}

In [46]:
reddit_corpus = ""
offset = 2400                                             #testing param only. this leaves you with the first 5 
                                                          #subreddits, with 1000 posts each for 5k posts

time1 = time.time()    
for i in range(0, len(subreddit_list) - 1 - offset):
    
    subreddit = subreddit_list[subreddit_id_to_title[i]]
    for post in subreddit:
        string_ripped_out = post['title'] + " " + post['text']
        reddit_corpus += string_ripped_out
        
time2 = time.time()
time_taken = time2 - time1
num = len(subreddit_list) - 1 - offset
length = num * 1000
print("Num Subreddits: {}".format(num))
print("Number of Posts: {}".format(length))
print("Time Taken: {} s".format(time_taken))

Num Subreddits: 98
Number of Posts: 98000
Time Taken: 1105.92920995 s


In [47]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences = tokenizer.tokenize(reddit_corpus)

In [48]:
def sentence_to_wordlist(raw):
    """
    convert to list of words
    remove unnecessary, split into words, no hyphens
    list of words
    """
    clean = re.sub("[^a-zA-z]", " ", raw)
    words = clean.split()
    return words

sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

token_count = sum([len(sentence) for sentence in sentences])
print(token_count)

4395892


In [63]:
reddit_vec = w2v.Word2Vec(
            sg=1,
            seed=1337,
            workers=8,
            size=300,
            min_count=3,
            window=12,
            sample=0.0001
        )

reddit_vec.build_vocab(sentences)

In [64]:
reddit_vec.train(sentences, total_examples=len(sentences), epochs=10)

23036853

In [65]:
reddit_vec.most_similar("Notch")

[(u'notch', 0.6171591281890869),
 (u'DCPU', 0.5765825510025024),
 (u'[source]', 0.5681474804878235),
 (u'dcpu', 0.5583937764167786),
 (u'Mojang', 0.5547876358032227),
 (u'Eldrone', 0.5356765985488892),
 (u'^c', 0.524700403213501),
 (u'ARG', 0.5212037563323975),
 (u'tweets', 0.520331084728241),
 (u'spacesuit', 0.5202186107635498)]

In [52]:
reddit_vec.most_similar(positive=['chrome','Mac'],negative=['PC'])

[(u'[Chrome', 0.5274796485900879),
 (u'aifhnlnghddfdaccgbbpbhjfkmncekmn', 0.5172223448753357),
 (u'Refresh]', 0.5120558738708496),
 (u'[ReloadEvery]', 0.5024615526199341),
 (u'springpad', 0.49507105350494385),
 (u'webstore', 0.48924514651298523),
 (u'firefox', 0.48251456022262573),
 (u'Chrome]', 0.47021985054016113),
 (u'Firefox', 0.4609581232070923),
 (u'ntp', 0.45537853240966797)]

In [66]:
len(reddit_vec.wv.vocab)

44177

In [67]:
reddit_vec.most_similar("xbox")

[(u'Bluetooth', 0.760131299495697),
 (u'adapters', 0.758177638053894),
 (u'VGA', 0.7518714666366577),
 (u'AVI', 0.7515074014663696),
 (u'macs', 0.7512806057929993),
 (u'Sixaxis', 0.7497506141662598),
 (u'NAS', 0.7466078996658325),
 (u'Logitech', 0.743848443031311),
 (u'DisplayPort', 0.7399234175682068),
 (u'DVI', 0.7378382086753845)]